In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pickle
import os
import pandas as pd
import json
from itertools import compress
import math

from scipy import signal 

import glob

from DataManager import ParticipantData, MeasurementData
from Helpers import read_normalized_json_to_df, save_to_disk, load_from_disk, create_rolling_windows, \
   intersection, reject_outliers, anglebetween, eye_outlier_removal_sigma, eye_outlier_removal_zero_values, validate_participant_seen_areas
from AngleHelpers import create_relative_directions

In [2]:
reference_data_all_failed_save_path = "./post_recorded_reference_data/reference_dict_all.pickle"
REFERENCE_DATA_ALL_EVENTS_FAILED = load_from_disk(reference_data_all_failed_save_path)

correlation_dict_path = "./post_recorded_reference_data/corr_dict.pickle"
CORRELATION_DICT = load_from_disk(correlation_dict_path)

# Load data

In [3]:
# Exclude all participants that have less than two (complete) scenes besides the training scene
exclude_participant_ids = [
    "a5c5d2051bd1493ea79e1d565524610f", # ['MountainRoad.txt']
    "2462e5d82bc44a328934442cae03d83f", # ['MountainRoad.txt', 'TrainingScene.txt']
    "0c2734d5a8664e1abd2ffcf8aa6fa5d6", # ['TrainingScene.txt']
    "c685e206b2eb4702860bb908afe2a11f", # ['MountainRoad.txt', 'TrainingScene.txt']
    "28d1f769c1204e36a574c025d8e771c4", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "d5f3fa1c877f4afb9bc0250d5882c22c", # ['TrainingScene.txt']
    "f4c4e380b2b941e4a963fe12f4038865", # ['MountainRoad.txt', 'TrainingScene.txt']
    "1c293d21f60241008d6230a6f28e4079", # ['TrainingScene.txt']
    "3e3154cff72540e48eac982a379ce69f", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "1deaa8e567f1435da07c0817717ef4fe", # ['TrainingScene.txt']
    "28191a22a70d478b808eb8905e35c6ba", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "ac30f09446ec48289c41b66b1ba0d3f9", # ['TrainingScene.txt'] 
    "5b79b725686a4985b5caf35de0e06d43", # ['Incomplete.txt', 'TrainingScene.txt']
    "b482259cbb524e689a6e3859951bebf1", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "518e07b76d804110b035e3c774f52d5b", # ['TrainingScene.txt']
    "cc9284dfb5f645858196537e05c1959b", # ['MountainRoad.txt']
    "0cd51e7040bc43568284cdcedccfe0d3", # ['TrainingScene.txt']
    "d9cd9704a0064fed9993333363608346", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "5ff5cecd56f7486286ec386434ae9219", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "178dd53ce37a40b0accd7a59cc9035ac", # ['TrainingScene.txt']
    "1efd4163b5614c8e9eed8e04c0ceda3e", # ['MountainRoad.txt', 'TrainingScene.txt'] 
    "b74d6990eba645fa82fa2ab9fb0b5180", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "711fa186ae5e4d9fada5b0e0049a2074", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "83b00eb628b2446d992ac698026dfdc0", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "f2fb2a62a3ca4033b9005be13842e27c", # ['MountainRoad.txt']
    "0c0c45348e314379994333127e5456ae", # ['TrainingScene.txt']
    "7824cc42ee0447a4a2fa56b9c93bbabb", # ['TrainingScene.txt']
    "95228b7595954054ae2d11cfbd7623f8", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "a5a31c4af00b4199ac4c8760bfaa469e", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "addd1085215d42b38184517a2dc95d8d", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "f4db5c989fa64b90b31d396943964c12", # ['TrainingScene.txt']
    "a33a91bbd197432c91a8156f88e46c08", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "206b42297be84c7087da6c93f46615bd", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "cf45d8bef0b842e5bac1b7022cc3c354", # ['MountainRoad.txt', 'TrainingScene.txt']
    "e9ec1e6c22c640168130d2756426666c", # ['TrainingScene.txt']
    "989685d9ca454491b184984695786e20", # ['TrainingScene.txt']
    "8f9b8786312349639e8811f451054df0", # ['MountainRoad.txt']
    "39e38c2aa06c42d79e50f36b631f9652", # ---- Some weird key error ---> files are empty for certain scenes
    "5e66bfb48366498aa83c364bf31c6059", # ---- Some weird key error
    "3358f368d8694472a86c4770d4f804bc", # ---- Some weird key error
    "95e8781012c94d37a222b5679724cb1d", # ---- Some weird key error
    "8626976ce8b14752be340be9f1070724", # ---- Some weird key error
    "ab79e465598844af94e9bfdfca116fc4", # ---- Some weird key error
    "6f7a67231a16454dbd657d2f48443aac", # ---- Some weird key error
    "2cae1d5a3c0c4872b2f397deb13ece71", # ---- Some weird key error
    "8b27ff8591af4cd79067cf156d23ab95", # ---- Some weird key error
    "8245ccbaba844032b08f7c33d9c4308a", # ---- Some weird key error 
    "d093379ce1224f1bb24b9730da105927", # ---- Some weird key error 
    "ff0cb5fdc1884204937eca64b52ee041", # ---- Some weird key error
    "05086cebdd4c45a084ed3ce6b25cc9e6", # ---- Some weird key error
    "4858c7aa9362421988e7c88d5adae755", # ---- Some weird key error
    "66bdb97a653b48f98c02d5e764089c00", # ---- Some weird key error
    "3100b15011da48d88fa3f7f053e4e19f", # ---- Some weird key error
    "39e5235b13274feb88430f08f3cd5369", # ---- Some weird key error
    "9b9220b8bf8c4e61adde8bc7571540ef", # ---- Some weird key error
    "a57132e2aeba484e8433ae8af9b1abbc", # ---- Some weird key error
    "3de6823af97441c18c8ca5f169b6e896", # ---- Some weird key error
    "3d865974f89e44c583e23e4f3c8f3d14", # ---- Some weird key error
    "011846a5c30c4325b45e439fc992f519", # ---- Some weird key error
    "951592742a88497989fbb71a65781639", # ---- Some weird key error 
    "867545b969cc4ff088b70eab632671dd", # ---- Some weird key error
    "0d72afe467e74d4fb717a4ab9bb128ac", # ---- Some weird key error
    "1d0296054e434021b3aa2b47b6c7d921", # ---- Number of supplied paths does not match 
    "0037b2329de444c18d751b4e79901b39", # ---- Calibration data seems to be missing
    "b89680ab48514db8bca8fcca1852f36d", # ---- Calibration data seems to be missing
    
]

In [5]:
verbose = True

participant_bootstrap_path = "/Volumes/LaCie/aMessdaten/LoopAR/Bootstrapped_Participants/"
keep_participants_in_memory = False


eyes_paths  = ["/Volumes/LaCie/aMessdaten/LoopAR/Data_from_store/LoopAR Data/WestdriveLoopARData_left_complete/EyeTracking/","/Volumes/LaCie/aMessdaten/LoopAR/Data_from_store/LoopAR Data/WestdriveLoopARData_right_complete/EyeTracking/"]
input_paths = ["/Volumes/LaCie/aMessdaten/LoopAR/Data_from_store/LoopAR Data/WestdriveLoopARData_left_complete/Input/","/Volumes/LaCie/aMessdaten/LoopAR/Data_from_store/LoopAR Data/WestdriveLoopARData_right_complete/Input/"]
calib_paths = ["/Volumes/LaCie/aMessdaten/LoopAR/Data_from_store/LoopAR Data/WestdriveLoopARData_left_complete/ParticipantCalibrationData/","/Volumes/LaCie/aMessdaten/LoopAR/Data_from_store/LoopAR Data/WestdriveLoopARData_right_complete/ParticipantCalibrationData/"]
scene_paths = ["/Volumes/LaCie/aMessdaten/LoopAR/Data_from_store/LoopAR Data/WestdriveLoopARData_left_complete/SceneData/","/Volumes/LaCie/aMessdaten/LoopAR/Data_from_store/LoopAR Data/WestdriveLoopARData_right_complete/SceneData/"]

eyes_paths  = ["./data/EyeTracking/"]
input_paths = ["./data/Input/"]
calib_paths = ["./data/ParticipantCalibrationData/"]
scene_paths = ["./data/SceneData/"]




#validate_participant_seen_areas(eyes_paths,exclude_participant_ids)
measurement1 = MeasurementData(eyes_paths,input_paths,calib_paths,scene_paths,participant_bootstrap_path,exclude_participant_ids,keep_participants_in_memory,verbose)


MeasurementData: Found files for participant 0bc17d704fec4a9eb892d905fc2e5da9.
Input data files: ['./data/Input/0bc17d704fec4a9eb892d905fc2e5da9_Input_Autobahn.txt', './data/Input/0bc17d704fec4a9eb892d905fc2e5da9_Input_Westbrueck.txt', './data/Input/0bc17d704fec4a9eb892d905fc2e5da9_Input_TrainingScene.txt', './data/Input/0bc17d704fec4a9eb892d905fc2e5da9_Input_MountainRoad.txt', './data/Input/0bc17d704fec4a9eb892d905fc2e5da9_Input_CountryRoad.txt']
Eyetracking data files: ['./data/EyeTracking/0bc17d704fec4a9eb892d905fc2e5da9_EyeTracking_MountainRoad.txt', './data/EyeTracking/0bc17d704fec4a9eb892d905fc2e5da9_EyeTracking_Westbrueck.txt', './data/EyeTracking/0bc17d704fec4a9eb892d905fc2e5da9_EyeTracking_TrainingScene.txt', './data/EyeTracking/0bc17d704fec4a9eb892d905fc2e5da9_EyeTracking_Autobahn.txt', './data/EyeTracking/0bc17d704fec4a9eb892d905fc2e5da9_EyeTracking_CountryRoad.txt']
Calibration data files: ['./data/ParticipantCalibrationData/0bc17d704fec4a9eb892d905fc2e5da9_ParticipantCalib

Area: Autobahn Segment: 3 Total datapoints (incl. resampled): 237 Filled NaNs: 103
Area: MountainRoad Segment: 0 Total datapoints (incl. resampled): 1992 Filled NaNs: 1041
Area: MountainRoad Segment: 1 Total datapoints (incl. resampled): 3148 Filled NaNs: 1644
Area: MountainRoad Segment: 2 Total datapoints (incl. resampled): 6041 Filled NaNs: 2630
Area: MountainRoad Segment: 3 Total datapoints (incl. resampled): 1223 Filled NaNs: 672
Area: CountryRoad Segment: 0 Total datapoints (incl. resampled): 2169 Filled NaNs: 1054
Area: CountryRoad Segment: 1 Total datapoints (incl. resampled): 3562 Filled NaNs: 1578
Area: CountryRoad Segment: 2 Total datapoints (incl. resampled): 2984 Filled NaNs: 1331
Area: CountryRoad Segment: 3 Total datapoints (incl. resampled): 4303 Filled NaNs: 1890
ParticipantData: Loaded data (raw) for participant 0d0d7bd79b1a48d4ad5e757cdbfc15c9.
ParticipantData: Saved bootstrapped data to disk.



In [12]:
#measurement1.measurement_data['0bc17d704fec4a9eb892d905fc2e5da9']["data"].golden_segment_data_vanilla
measurement1.get_data(use_vanilla=False, filter_data=True,filter_by_corr_coeff_dict = CORRELATION_DICT, corr_coeff_threshold=0.95)

ParticipantData: Initialising participant 0bc17d704fec4a9eb892d905fc2e5da9.
ParticipantData: Loaded data (bootstrapped) for participant 0bc17d704fec4a9eb892d905fc2e5da9.
ParticipantData: Filtering data by Correlation Coefficients with threshold 0.95.
ParticipantData: Corr-coeff filtering excluded MountainRoad segment 1 with min corr-coeffs 0.9440682544132852.
ParticipantData: Corr-coeff filtering excluded MountainRoad segment 2 with min corr-coeffs 0.4288595845461723.
ParticipantData: Corr-coeff filtering excluded Westbrueck segment 2 with min corr-coeffs 0.9055197317583018.
ParticipantData: Corr-coeff filtering excluded Autobahn segment 1 with min corr-coeffs 0.6856464347733652.
ParticipantData: Corr-coeff filtering excluded Autobahn segment 3 with min corr-coeffs -0.9139574552395952.
ParticipantData: Corr-coeff filtering excluded CountryRoad segment 1 with min corr-coeffs 0.9378459305925257.
ParticipantData: Corr-coeff filtering excluded CountryRoad segment 2 with min corr-coeffs 0.8

{'0bc17d704fec4a9eb892d905fc2e5da9': {'MountainRoad': {0:       HmdPosition.x  HmdPosition.y  HmdPosition.z  NoseVector.x  NoseVector.y  \
   0         -0.147156       1.009186       0.414246      0.006151      0.053623   
   1         -0.147176       1.009176       0.414246      0.006049      0.053482   
   2         -0.147196       1.009165       0.414246      0.005946      0.053342   
   3         -0.147217       1.009155       0.414246      0.005843      0.053202   
   4         -0.147186       1.009155       0.414261      0.005617      0.053287   
   ...             ...            ...            ...           ...           ...   
   1686      -0.318817       1.058548       0.340286     -0.077524      0.072861   
   1687      -0.489197       1.073425       0.303589     -0.078941      0.073112   
   1688      -0.546895       1.078079       0.290802     -0.080144      0.073172   
   1689      -0.604594       1.082733       0.278015     -0.081347      0.073232   
   1690      -0.49646

In [7]:
print(measurement1.get_total_available_areas())
measurement1.get_participant_list()

{'Westbrueck': 2, 'CountryRoad': 2, 'MountainRoad': 2, 'Autobahn': 2}


['0bc17d704fec4a9eb892d905fc2e5da9', '0d0d7bd79b1a48d4ad5e757cdbfc15c9']

In [7]:
print(1/0)

ZeroDivisionError: division by zero

In [8]:
measurement1.apply_reference_data(REFERENCE_DATA_ALL_EVENTS_FAILED)

MeasurementData: Applying reference data to all participants individually...
ParticipantData: Initialising participant 0bc17d704fec4a9eb892d905fc2e5da9.


 35%|███▍      | 587/1691 [00:00<00:00, 5867.71it/s]

ParticipantData: Loaded data (bootstrapped) for participant 0bc17d704fec4a9eb892d905fc2e5da9.
ParticipantData: Applying reference data to back of MountainRoad's segment 0. Number of datapoints used: 1691.
Creating relative directions considering roll.


 40%|████      | 681/1691 [00:00<00:00, 6807.47it/s]

Creating relative directions considering roll.


...

ParticipantData: Applying reference data to back of CountryRoad's segment 3. Number of datapoints used: 4181.
Creating relative directions considering roll.


 31%|███       | 1288/4181 [00:00<00:00, 6472.73it/s]

Creating relative directions considering roll.


100%|██████████| 4181/4181 [00:00<00:00, 5794.62it/s]


ParticipantData: Saved bootstrapped data to disk.

ParticipantData: Updated bootstrapped data on disk.
MeasurementData: Done applying reference data to all participants individually.


In [9]:
print("Available participants: \n" + str(measurement1.get_participant_list()))
print()

Available participants: 
['0bc17d704fec4a9eb892d905fc2e5da9', '0d0d7bd79b1a48d4ad5e757cdbfc15c9']



# Extract participant data

In [ ]:
### Reference data applied 

# Extract data of one participant, of one area, of one segment 
participant_exclusion_list = [p for p in measurement1.get_participant_list() if p != GLOBAL_SINGLE_PARTICIPANT]
one_participant_single_segment = measurement1.get_data(use_vanilla=False, filter_data=True, \
                 filter_by_corr_coeff_dict=CORRELATION_DICT, corr_coeff_threshold=0.95, \
                 exclude_participants=participant_exclusion_list)
one_participant_single_segment = one_participant_single_segment[GLOBAL_SINGLE_PARTICIPANT][GLOBAL_AREA][GLOBAL_SEGMENT] 

# Average data over multiple sessions per segment
average_over_all_participants = measurement1.get_data(use_vanilla = False, filter_data=True, filter_by_corr_coeff_dict=CORRELATION_DICT, corr_coeff_threshold=0.95, exclude_participants=[])
average_over_all_participants = measurement1.average_data(average_over_all_participants)
average_over_all_participants_single_segment = average_over_all_participants[GLOBAL_AREA][GLOBAL_SEGMENT] 


In [ ]:
### No reference data applied 

# Extract data of one participant, of one area, of one segment without ref data applied 
participant_exclusion_list = [p for p in measurement1.get_participant_list() if p != GLOBAL_SINGLE_PARTICIPANT]
one_participant_single_segment_no_ref_applied = measurement1.get_data(use_vanilla=True, filter_data=True, \
                 filter_by_corr_coeff_dict=CORRELATION_DICT, corr_coeff_threshold=0.95, \
                 exclude_participants=participant_exclusion_list)
one_participant_single_segment_no_ref_applied = \
    one_participant_single_segment_no_ref_applied[GLOBAL_SINGLE_PARTICIPANT][GLOBAL_AREA][GLOBAL_SEGMENT]

# Extract reference data for one area, one segment 
ref_data_one_participant_single_segment = REFERENCE_DATA_ALL_EVENTS_FAILED[GLOBAL_AREA][GLOBAL_SEGMENT]  


In [ ]:
print('X min/max ref: ' + str(round(one_participant_single_segment['EyeDirWorldCombined.x'].min(), 4)) + ', ' + str(round(one_participant_single_segment['EyeDirWorldCombined.x'].max(), 4)))
print('Y min/max ref: ' + str(round(one_participant_single_segment['EyeDirWorldCombined.y'].min(), 4)) + ', ' + str(round(one_participant_single_segment['EyeDirWorldCombined.y'].max(), 4)))
print('Z min/max ref: ' + str(round(one_participant_single_segment['EyeDirWorldCombined.z'].min(), 4)) + ', ' + str(round(one_participant_single_segment['EyeDirWorldCombined.z'].max(), 4)))

print('X min/max no ref: ' + str(round(one_participant_single_segment_no_ref_applied['EyeDirWorldCombined.x'].min(), 4)) + ', ' + str(round(one_participant_single_segment_no_ref_applied['EyeDirWorldCombined.x'].max(), 4)))
print('Y min/max no ref: ' + str(round(one_participant_single_segment_no_ref_applied['EyeDirWorldCombined.y'].min(), 4)) + ', ' + str(round(one_participant_single_segment_no_ref_applied['EyeDirWorldCombined.y'].max(), 4)))
print('Z min/max no ref: ' + str(round(one_participant_single_segment_no_ref_applied['EyeDirWorldCombined.z'].min(), 4)) + ', ' + str(round(one_participant_single_segment_no_ref_applied['EyeDirWorldCombined.z'].max(), 4)))

In [ ]:
one_participant_single_segment

# Check unit length of directions